## Import libraries

In [1]:
import fitz
import json
from re import findall
from re import sub
from os import listdir
from os.path import isfile, join
import PyPDF2
import glob
import en_core_web_sm
import numpy as np  
import pandas as pd
import re  
import nltk  
from sklearn.datasets import load_files 
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
nltk.download('stopwords')  
import pickle  
from nltk.corpus import stopwords
import string
from operator import itemgetter


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abdullah.kurkcu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Reading Resumes into DataFrame

Does it have education?
Does it have professional?
Does it have project experience?
Does it have professional associations?

In [2]:
# All the pdf files in the folder
pdf_files = glob.glob("%s/*.pdf" % "../resumes/")

i = 0 # 81 is the start of not resume
styles = {}
for file in pdf_files:
    doc = fitz.open(file)
    for page in doc:
        blocks = page.getText("dict")["blocks"]
        fonts = []
        sizes = []
        colors = []
        images = []
        body = ''
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # block contains text
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        body += " " + s['text'].lower()
                        fonts.append(s['font'])
                        sizes.append(s['size'])
                        colors.append(s['color'])
            if b['type'] == 1:  # block contains image
                images.append(1)
        if 'education' in body:
            ed_check = 1
        else:
            ed_check = 0
        if 'professional registrations' in body:
            pr_check = 1
        else:
            pr_check = 0
        if 'associations' in body:
            pa_check = 1
        else:
            pa_check = 0
        if 'experience' in body:
            pe_check = 1
        else:
            pe_check = 0
        if len(images)>0:
            img_check = 1
        else:
            img_check = 0
        
    if i < 81:
        label = 'resume'
    else:
        label = 'not resume'
    styles[doc] =  {'name':doc, 'body':body,'fonts':list(set(fonts)), 'size':list(set(sizes)),'color':list(set(colors)),'education':ed_check, 'registrations':pr_check,
                   'associations':pa_check, 'experience':pe_check, 'image':img_check, 'label':label}
    i = i + 1

data = pd.DataFrame.from_dict(styles, orient='index')

mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object


#### Notes
Since it transforms fonts and sizes to columns, new documents will not have the same amount of columns that the test data will have

In [124]:
data = data.reset_index(drop=True)
mlb = MultiLabelBinarizer()
df = pd.DataFrame(mlb.fit_transform(data['fonts']),columns=mlb.classes_, index=data.index)
df2 = pd.DataFrame(mlb.fit_transform(data['size']),columns=mlb.classes_, index=data.index)
df3 = pd.DataFrame(mlb.fit_transform(data['color']),columns=mlb.classes_, index=data.index)
ml_data = ((df.join(df2)).join(df3)).join(data)
ml_data = ml_data.drop(columns=['name','fonts','size','color','body'])
label = ml_data.iloc[:,-1]
train = ml_data.iloc[:,1:-2]
resumes_train, resumes_test, y_train, y_test = train_test_split(train, label, test_size=0.33, random_state=42)

In [125]:
train

,Arial-BoldMT,Arial-ItalicMT,ArialMT,ArialNarrow,"ArialNarrow,Bold",ArialNarrow-Bold,ArialNarrow-Italic,AvenirLTStd-Book,AvenirNextLTPro-Bold,AvenirNextLTPro-BoldCn,...,13027530,14687781,15562019,16296474,16711680,16777215,education,registrations,associations,experience
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
113,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
114,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
115,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [113]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB

# Using Decision Tree Classifier on the data
dtclf = tree.DecisionTreeClassifier()
dtclf = dtclf.fit(resumes_train, y_train)
print(dtclf.score(resumes_test, y_test))


# Using Random Forest Classifier on the data
rfclf = RandomForestClassifier()
rfclf = rfclf.fit(resumes_train, y_train)
print(rfclf.score(resumes_test, y_test))

# Using SVM Classifier on the data
model_svm = svm.SVC()
model_svm = model_svm.fit(resumes_train, y_train)
print(model_svm.score(resumes_test, y_test))

# Using Bernoulli Naive Bayes Algorithm
bnbclf = BernoulliNB()
bnbclf = bnbclf.fit(resumes_train, y_train)
print(bnbclf.score(resumes_test, y_test))

# Using Gaussian Naive Bayes Algorithm
gnbclf = GaussianNB()
gnbclf = gnbclf.fit(resumes_train, y_train)
print(gnbclf.score(resumes_test, y_test))

0.9743589743589743
1.0
0.9743589743589743
0.9743589743589743
1.0


In [127]:
styles = {}

file = '..//downloaded//20.01139 - JeffCo  Prelim-Final Eng Svcs - S Kipling Pkwy - Final.pdf'
doc = fitz.open(file)
for page in doc:
    blocks = page.getText("dict")["blocks"]
    fonts = []
    sizes = []
    colors = []
    images = []
    body = ''
    for b in blocks:  # iterate through the text blocks
        if b['type'] == 0:  # block contains text
            for l in b["lines"]:  # iterate through the text lines
                for s in l["spans"]:  # iterate through the text spans
                    body += " " + s['text'].lower()
                    fonts.append(s['font'])
                    sizes.append(s['size'])
                    colors.append(s['color'])
        if b['type'] == 1:  # block contains image
            images.append(1)
    if 'education' in body:
        ed_check = 1
    else:
        ed_check = 0
    if 'professional registrations' in body:
        pr_check = 1
    else:
        pr_check = 0
    if 'associations' in body:
        pa_check = 1
    else:
        pa_check = 0
    if 'experience' in body:
        pe_check = 1
    else:
        pe_check = 0
    if len(images)>0:
        img_check = 1
    else:
        img_check = 0

if i < 81:
    label = 'resume'
else:
    label = 'not resume'
styles[doc] =  {'name':doc, 'body':body,'fonts':list(set(fonts)), 'size':list(set(sizes)),'color':list(set(colors)),'education':ed_check, 'registrations':pr_check,
               'associations':pa_check, 'experience':pe_check, 'image':img_check, 'label':label}
data = pd.DataFrame.from_dict(styles, orient='index')
data = data.reset_index(drop=True)
mlb = MultiLabelBinarizer()
df = pd.DataFrame(mlb.fit_transform(data['fonts']),columns=mlb.classes_, index=data.index)
df2 = pd.DataFrame(mlb.fit_transform(data['size']),columns=mlb.classes_, index=data.index)
df3 = pd.DataFrame(mlb.fit_transform(data['color']),columns=mlb.classes_, index=data.index)
ml_data = ((df.join(df2)).join(df3)).join(data)
ml_data = ml_data.drop(columns=['name','fonts','size','color','body'])
label = ml_data.iloc[:,-1]
train = ml_data.iloc[:,1:-2]


In [148]:
ml_data

,Arial,"Arial,Bold",Courier,Garamond,TimesNewRoman,6.72845458984375,7.4460530281066895,8.253531455993652,8.971373558044434,0,8355711,education,registrations,associations,experience,image,label
0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,not resume


In [150]:
df = ml_data2.append(ml_data[:,1:-2]).fillna(0)

In [153]:
df

,Arial,Arial-BoldMT,Arial-ItalicMT,ArialMT,ArialNarrow,"ArialNarrow,Bold",ArialNarrow-Bold,ArialNarrow-Italic,AvenirLTStd-Book,AvenirNextLTPro-Bold,...,experience,image,label,"Arial,Bold",Garamond,6.72845458984375,7.4460530281066895,8.253531455993652,8.971373558044434,8355711
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,resume,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,resume,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,resume,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,resume,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,resume,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,not resume,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,not resume,0.0,0.0,0.0,0.0,0.0,0.0,0.0
115,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,not resume,0.0,0.0,0.0,0.0,0.0,0.0,0.0
116,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,not resume,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [154]:
ml_data2.loc[0]
train = df.iloc[:,1:-2]

In [159]:
train = train.drop(columns='label')

In [161]:
train

,Arial-BoldMT,Arial-ItalicMT,ArialMT,ArialNarrow,"ArialNarrow,Bold",ArialNarrow-Bold,ArialNarrow-Italic,AvenirLTStd-Book,AvenirNextLTPro-Bold,AvenirNextLTPro-BoldCn,...,education,registrations,associations,experience,image,"Arial,Bold",Garamond,6.72845458984375,7.4460530281066895,8.253531455993652
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,1,1,1,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,0,1,1,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,0,1,1,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,0,1,1,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,1,1,1,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,1,0.0,0.0,0.0,0.0,0.0
114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0
115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0
116,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,1,0.0,0.0,0.0,0.0,0.0


In [126]:
ml_data2 = ml_data

In [160]:
gnbclf.predict(train)

ValueError: operands could not be broadcast together with shapes (118,239) (233,) 

In [98]:
numeric_features = clist
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['body']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression())])

clf.fit(resumes_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

ValueError: No valid specification of the columns. Only a scalar, list or slice of all integers or all strings, or boolean mask is allowed

In [108]:
resumes_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78 entries, 67 to 102
Columns: 233 entries, Arial-BoldMT to experience
dtypes: int32(229), int64(4)
memory usage: 72.8 KB


## Convert text into a vector 

In [99]:
from sklearn.model_selection import train_test_split  
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1,2))


# Do not call fit transform twice because it is stored in vectorizer 
X_train = vectorizer.fit_transform(resumes_train)
X_test = vectorizer.transform(resumes_test)
y_train = y_train

AttributeError: 'float' object has no attribute 'lower'

In [8]:
X_train

<10x10 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in Compressed Sparse Row format>

## Train Classifier

In [7]:
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)  
text_clf  = classifier.fit(X_train, y_train) 

ValueError: Number of labels=78 does not match number of samples=10

In [98]:
file = '../downloaded/20.02373 CDOT NPS Western Slope_Ulteig.pdf'
fileReader = PyPDF2.PdfFileReader(open(file,'rb'))
count = fileReader.numPages


results = text_clf.predict(X_test)
resumesTest['results'] = results
results

array(['NotResume', 'NotResume', 'NotResume', 'NotResume', 'NotResume',
       'NotResume', 'NotResume', 'NotResume', 'NotResume', 'NotResume',
       'NotResume', 'NotResume', 'NotResume', 'NotResume', 'NotResume',
       'NotResume', 'NotResume', 'NotResume', 'NotResume', 'NotResume',
       'NotResume', 'NotResume', 'NotResume', 'NotResume', 'NotResume',
       'NotResume', 'NotResume', 'Resume', 'Resume', 'Resume', 'Resume',
       'Resume', 'Resume', 'Resume', 'Resume', 'Resume', 'Resume',
       'Resume', 'Resume', 'Resume', 'Resume', 'Resume', 'Resume',
       'Resume', 'Resume', 'Resume', 'Resume', 'Resume', 'Resume'],
      dtype=object)

In [70]:
text = ""
i = 0
X_test = []
while i < count:
    text = fileReader.getPage(i).extractText()
    X_test.append(text)
    
    i = i +1  



X_test = vectorizer.transform(X_test)

## Make a prediction

In [65]:
resumesTest.loc[0]['text']

'proposal to perform professional engineering services for  gore valley trail design improvements prepared for town of vail angy casamento peangycasamentoulteigcom       february  '

### Other

In [52]:
# All the pdf files in the folder
pdf_files = glob.glob("%s/*.pdf" % "../resumes/")


# Resumes -> convert to a dictionary with an employee name
resumes = pd.DataFrame()

texts = []
# File loop
for file in pdf_files:
    print(file)
    text = ""
    try:
        fileReader = PyPDF2.PdfFileReader(open(file,'rb'))
        count = fileReader.numPages
        text = ""
        i = 0
        while i < count:
            text += fileReader.getPage(i).extractText()
            i = i +1  
        # lowercase
        text = text.lower()
        # remove nubmers
        text = re.sub(r'\d+', '', text)
        # remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))
        # remove \n
        text = text.replace('\n',' ')
        # remove ndł
        text= text.replace('ndł',' ')
        # remove \nł
        text = text.replace('\nł',' ')
        # remove ł
        text = text.replace('ł',' ')
        # remove \nł
        text = text.replace('\nł',' ')

        texts.append(text)
    except Exception as e:
        print(e)
        pass
    
resumes['text'] = texts
resumes['classification'] = "Resume"

../resumes\17.00532 0.pdf
../resumes\17.00532 1.pdf
../resumes\17.00532 2.pdf
../resumes\17.00532 3.pdf
../resumes\18.00308 0.pdf
../resumes\18.00308 1.pdf
../resumes\18.00308 2.pdf
../resumes\18.00308 3.pdf
../resumes\19.00064 0.pdf
../resumes\19.00064 1.pdf
../resumes\19.00064 10.pdf
../resumes\19.00064 11.pdf
../resumes\19.00064 2.pdf
../resumes\19.00064 3.pdf
../resumes\19.00064 4.pdf
../resumes\19.00064 5.pdf
../resumes\19.00064 6.pdf
../resumes\19.00064 7.pdf
../resumes\19.00064 8.pdf
../resumes\19.00064 9.pdf
../resumes\20.00432 0.pdf
../resumes\20.00432 1.pdf
../resumes\20.00432 2.pdf
../resumes\20.00432 3.pdf
../resumes\20.00432 4.pdf
../resumes\20.00973 0.pdf
../resumes\20.00973 1.pdf
../resumes\20.00973 10.pdf
../resumes\20.00973 11.pdf
../resumes\20.00973 12.pdf
../resumes\20.00973 2.pdf
../resumes\20.00973 3.pdf
../resumes\20.00973 4.pdf
../resumes\20.00973 5.pdf
../resumes\20.00973 6.pdf
../resumes\20.00973 7.pdf
../resumes\20.00973 8.pdf
../resumes\20.00973 9.pdf
../resu

In [185]:
from operator import itemgetter
def fonts(doc, granularity=False):
    """Extracts fonts and their usage in PDF documents.
    :param doc: PDF document to iterate through
    :type doc: <class 'fitz.fitz.Document'>
    :param granularity: also use 'font', 'flags' and 'color' to discriminate text
    :type granularity: bool
    :rtype: [(font_size, count), (font_size, count}], dict
    :return: most used fonts sorted by count, font style information
    """
    styles = {}
    font_counts = {}

    for page in doc:
        blocks = page.getText("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # block contains text
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if granularity:
                            styles[s['font']] =  {'size':s['size'],'color':s['color']}
                        else:
                            styles[s['font']] =  {'size':s['size'],'color':s['color']}

    return styles

### OpenCV trials

In [1]:
from pdf2image import convert_from_path

file = '../downloaded/20.02373 CDOT NPS Western Slope_Ulteig.pdf'
images = convert_from_path(file)

In [6]:
# All the pdf files in the folder
pdf_files = glob.glob("%s/*.pdf" % "../resumes/")

# File loop
i = 0 
for file in pdf_files:
    images = convert_from_path(file)
    for image in images:
        filename = "resume" + str(i) + ".jpg"
        image.save(filename, 'JPEG')
        i = i +1